# IBM Capstone Project
---


This notebook is meant to support the Predicting Accident Severity report in the same github repo. 

It follows the same logical flow :

- Problem (omitted)
- Methodology
- Results
- Discussion (omitted)
- Conclusion (omitted)

In [1]:
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import warnings
import datetime
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

warnings.filterwarnings('ignore')

## Methodology
### Data Understanding

In [2]:
collisions_df = pd.read_csv("Data-Collisions.csv")
pd.set_option('display.max_columns', None)
collisions_df.head()

,SEVERITYCODE,X,Y,OBJECTID,INCKEY,COLDETKEY,REPORTNO,STATUS,ADDRTYPE,INTKEY,LOCATION,EXCEPTRSNCODE,EXCEPTRSNDESC,SEVERITYCODE.1,SEVERITYDESC,COLLISIONTYPE,PERSONCOUNT,PEDCOUNT,PEDCYLCOUNT,VEHCOUNT,INCDATE,INCDTTM,JUNCTIONTYPE,SDOT_COLCODE,SDOT_COLDESC,INATTENTIONIND,UNDERINFL,WEATHER,ROADCOND,LIGHTCOND,PEDROWNOTGRNT,SDOTCOLNUM,SPEEDING,ST_COLCODE,ST_COLDESC,SEGLANEKEY,CROSSWALKKEY,HITPARKEDCAR
0,2,-122.323148,47.703140,1,1307,1307,3502005,Matched,Intersection,37475.0,5TH AVE NE AND NE 103RD ST,,NaN,2,Injury Collision,Angles,2,0,0,2,2013/03/27 00:00:00+00,3/27/2013 2:54:00 PM,At Intersection (intersection related),11,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, FRONT END ...",NaN,N,Overcast,Wet,Daylight,NaN,NaN,NaN,10,Entering at angle,0,0,N
1,1,-122.347294,47.647172,2,52200,52200,2607959,Matched,Block,NaN,AURORA BR BETWEEN RAYE ST AND BRIDGE WAY N,NaN,NaN,1,Property Damage Only Collision,Sideswipe,2,0,0,2,2006/12/20 00:00:00+00,12/20/2006 6:55:00 PM,Mid-Block (not related to intersection),16,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, LEFT SIDE ...",NaN,0,Raining,Wet,Dark - Street Lights On,NaN,6354039.0,NaN,11,From same direction - both going straight - bo...,0,0,N
2,1,-122.334540,47.607871,3,26700,26700,1482393,Matched,Block,NaN,4TH AVE BETWEEN SENECA ST AND UNIVERSITY ST,NaN,NaN,1,Property Damage Only Collision,Parked Car,4,0,0,3,2004/11/18 00:00:00+00,11/18/2004 10:20:00 AM,Mid-Block (not related to intersection),14,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, REAR END",NaN,0,Overcast,Dry,Daylight,NaN,4323031.0,NaN,32,One parked--one moving,0,0,N
3,1,-122.334803,47.604803,4,1144,1144,3503937,Matched,Block,NaN,2ND AVE BETWEEN MARION ST AND MADISON ST,,NaN,1,Property Damage Only Collision,Other,3,0,0,3,2013/03/29 00:00:00+00,3/29/2013 9:26:00 AM,Mid-Block (not related to intersection),11,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, FRONT END ...",NaN,N,Clear,Dry,Daylight,NaN,NaN,NaN,23,From same direction - all others,0,0,N
4,2,-122.306426,47.545739,5,17700,17700,1807429,Matched,Intersection,34387.0,SWIFT AVE S AND SWIFT AV OFF RP,NaN,NaN,2,Injury Collision,Angles,2,0,0,2,2004/01/28 00:00:00+00,1/28/2004 8:04:00 AM,At Intersection (intersection related),11,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, FRONT END ...",NaN,0,Raining,Wet,Daylight,NaN,4028032.0,NaN,10,Entering at angle,0,0,N


In [3]:
collisions_df.shape

(194673, 38)

### Data Preparation

In [4]:
#Removing unused attributes
collisions_df_d = collisions_df.drop(columns=['X','Y','OBJECTID', 'INCKEY', 'COLDETKEY', 'REPORTNO',
       'STATUS', 'ADDRTYPE', 'INTKEY', 'LOCATION', 'EXCEPTRSNCODE',
       'EXCEPTRSNDESC', 'SEVERITYCODE.1', 'SEVERITYDESC', 'COLLISIONTYPE','SPEEDING','INATTENTIONIND',"UNDERINFL",
       'PERSONCOUNT', 'PEDCOUNT', 'PEDCYLCOUNT', 'VEHCOUNT', 'JUNCTIONTYPE', 'SDOT_COLCODE', 'SDOT_COLDESC',
       'PEDROWNOTGRNT', 'SDOTCOLNUM', 'ST_COLCODE', 'ST_COLDESC',
       'SEGLANEKEY', 'CROSSWALKKEY', 'HITPARKEDCAR'])

In [5]:
#Resulting dataframe
collisions_df_d

,SEVERITYCODE,INCDATE,INCDTTM,WEATHER,ROADCOND,LIGHTCOND
0,2,2013/03/27 00:00:00+00,3/27/2013 2:54:00 PM,Overcast,Wet,Daylight
1,1,2006/12/20 00:00:00+00,12/20/2006 6:55:00 PM,Raining,Wet,Dark - Street Lights On
2,1,2004/11/18 00:00:00+00,11/18/2004 10:20:00 AM,Overcast,Dry,Daylight
3,1,2013/03/29 00:00:00+00,3/29/2013 9:26:00 AM,Clear,Dry,Daylight
4,2,2004/01/28 00:00:00+00,1/28/2004 8:04:00 AM,Raining,Wet,Daylight
...,...,...,...,...,...,...
194668,2,2018/11/12 00:00:00+00,11/12/2018 8:12:00 AM,Clear,Dry,Daylight
194669,1,2018/12/18 00:00:00+00,12/18/2018 9:14:00 AM,Raining,Wet,Daylight
194670,2,2019/01/19 00:00:00+00,1/19/2019 9:25:00 AM,Clear,Dry,Daylight
194671,2,2019/01/15 00:00:00+00,1/15/2019 4:48:00 PM,Clear,Dry,Dusk


In [6]:
#Checking the type
collisions_df_d.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 194673 entries, 0 to 194672
Data columns (total 6 columns):
SEVERITYCODE    194673 non-null int64
INCDATE         194673 non-null object
INCDTTM         194673 non-null object
WEATHER         189592 non-null object
ROADCOND        189661 non-null object
LIGHTCOND       189503 non-null object
dtypes: int64(1), object(5)
memory usage: 8.9+ MB


In [7]:
#Checking the number of nulls
collisions_df_d.isnull().any(axis=1).value_counts(normalize=True)

False    0.97259
True     0.02741
dtype: float64

In [8]:
#Target Imbalance check
collisions_df.SEVERITYCODE.value_counts(normalize=True)

1    0.701099
2    0.298901
Name: SEVERITYCODE, dtype: float64

In [9]:
#Changing the type & Filling null values
collisions_df_d.SEVERITYCODE = collisions_df.SEVERITYCODE.replace([2,1],[1,0]).astype(int)
collisions_df_d.INCDTTM = collisions_df.INCDTTM.astype('datetime64[ns]').dt.hour
collisions_df_d.INCDATE = collisions_df.INCDATE.astype('datetime64[ns]').dt.dayofweek
collisions_df_d.WEATHER = collisions_df.WEATHER.replace(["Unknown","Other"],np.nan).fillna("Clear").astype("category")
collisions_df_d.ROADCOND = collisions_df.ROADCOND.replace(["Unknown","Other"],np.nan).fillna("Dry").astype("category")
collisions_df_d.LIGHTCOND = collisions_df.LIGHTCOND.replace(["Unknown","Other"],np.nan).fillna("Daylight").astype("category")

In [10]:
#Preparing for encoding 1/3
weather_dic ={"Clear":0,
              "Partly Cloudy":0.5,
              "Overcast":1,
              "Blowing Sand/Dirt":1.5,
              "Severe Crosswind":2,
              "Raining":2,
              "Sleet/Hail/Freezing Rain":2.5,
              "Snowing":3,
              "Fog/Smog/Smoke":3,
             }

In [11]:
#Preparing for encoding 2/3
road_dic ={"Dry":0,
           "Wet":1,
           "Sand/Mud/Dirt":1,
           "Snow/Slush":1.5,
           "Oil":1.5,
           "Standing Water":2,
           "Ice":3,
           }

In [12]:
#Preparing for encoding 3/3
light_dic ={"Daylight":0,
           "Dark - Street Lights On":1.5,
           "Dawn":1,
           "Dusk":1,
           "Dark - Unknown Lighting":2,
           "Dark - Street Lights Off":3,
           "Dark - No Street Lights":3,
           }

In [13]:
#Encoding the dataframe
collisions_df_gd = pd.concat([collisions_df_d[["SEVERITYCODE","INCDATE","INCDTTM"]],
                           collisions_df_d['WEATHER'].map(weather_dic).astype(int),
                           collisions_df_d['ROADCOND'].map(road_dic).astype(int),
                           collisions_df_d['LIGHTCOND'].map(light_dic).astype(int)],
                           axis=1)

### Modelling

In [14]:
X = collisions_df_gd.iloc[:,1:]
y = collisions_df_gd.iloc[:,0]

In [15]:
from sklearn.model_selection import train_test_split

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=42)

In [17]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.dummy import DummyClassifier

from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import jaccard_score

np.set_printoptions(precision=2)

In [18]:
#Creating a list of classifier for future iteration
classifiers = [
    DummyClassifier(strategy="most_frequent"),
    KNeighborsClassifier(3),
    LogisticRegression(random_state=0,class_weight="balanced"),
    DecisionTreeClassifier(max_depth=5,class_weight='balanced'),
    ]
names = ["Dummy Classifier","Nearest Neighbors", "Logistic Regression", "Decision Tree"]

## Results

In [19]:
#Iterating over the classifiers and returning test scores
scores = []
for name, clf in zip(names, classifiers):
        clf.fit(X_train, y_train)
        score = clf.score(X_test, y_test)
        f1 = f1_score(clf.predict(X_test), y_test)
        jac = jaccard_score(clf.predict(X_test), y_test)
        scores.append([score,f1,jac])
            
scores_df = pd.DataFrame(scores,columns = ["Accuracy","F1","Jaccard"],index = names)
scores_df

,Accuracy,F1,Jaccard
Dummy Classifier,0.706030,0.000000,0.000000
Nearest Neighbors,0.618708,0.273039,0.158104
Logistic Regression,0.528354,0.395444,0.246451
Decision Tree,0.496302,0.436566,0.279236
